# Predictions

In the main notebook, we trained different models, where the dependent variable `estimated_total_comprehensive_cost`. In this notebook, we will use that model to predict based on new data.

In [1]:
from utils.utils import Pickler
import pandas as pd
import numpy as np

In [2]:
models = Pickler('./data/models.pkl').load()
numeric_cols, categorical_cols, scaler = Pickler('./data/scaler.pkl').load()

# Example

In [3]:
df = pd.read_csv('./data/austin_car_crash_cleaned.csv')

cols = ['fatal_crash', 'speed_limit', 'sus_serious_injry_cnt', 'nonincap_injry_cnt', 'poss_injry_cnt', 'non_injry_cnt', 'unkn_injry_cnt', 'motor_vehicle_death_count', 'motor_vehicle_serious_injury_count', 'bicycle_death_count', 'bicycle_serious_injury_count', 'pedestrian_death_count', 'pedestrian_serious_injury_count', 'motorcycle_death_count', 'motorcycle_serious_injury_count', 'other_death_count', 'other_serious_injury_count', 'micromobility_serious_injury_count', 'micromobility_death_count', 'severity_killed', 'severity_not_injured', 'severity_unknown', 'unit_involved_motor_vehicle_other', 'unit_involved_motorcycle', 'hour', 'weekend', 'hour_cos']

example = df[cols]
print(example.iloc[0])

fatal_crash                              False
speed_limit                               35.0
sus_serious_injry_cnt                        0
nonincap_injry_cnt                           0
poss_injry_cnt                               4
non_injry_cnt                                1
unkn_injry_cnt                               0
motor_vehicle_death_count                    0
motor_vehicle_serious_injury_count           0
bicycle_death_count                          0
bicycle_serious_injury_count                 0
pedestrian_death_count                       0
pedestrian_serious_injury_count              0
motorcycle_death_count                       0
motorcycle_serious_injury_count              0
other_death_count                            0
other_serious_injury_count                   0
micromobility_serious_injury_count           0
micromobility_death_count                    0
severity_killed                          False
severity_not_injured                     False
severity_unkn

In [4]:
print(df['estimated_total_comprehensive_cost'].iloc[0])

820000


# New Independent Variables

The models were trained with 21 independent variables.

- fatal_crash
- speed_limit
- nonincap_injry_cnt
- poss_injry_cnt
- non_injry_cnt
- unkn_injry_cnt
- tot_injry_cnt
- death_cnt
- pedestrian_death_count
- motorcycle_death_count
- other_death_count
- micromobility_death_count
- estimated_total_comprehensive_cost
- severity_non_incapacitating_injury
- severity_possible_injury
- severity_unknown
- unit_involved_motor_vehicle_other
- unit_involved_pedestrian
- hour
- weekend
- hour_cos


In [5]:
vars = {
    'fatal_crash': False,
    'speed_limit': 35.0,
    'sus_serious_injry_cnt': 0,
    'nonincap_injry_cnt': 0,
    'poss_injry_cnt': 4,
    'non_injry_cnt': 1,
    'unkn_injry_cnt': 0,
    'motor_vehicle_death_count': 0,
    'motor_vehicle_serious_injury_count': 0,
    'bicycle_death_count': 0,
    'bicycle_serious_injury_count': 0,
    'pedestrian_death_count': 0,
    'pedestrian_serious_injury_count': 0,
    'motorcycle_death_count': 0,
    'motorcycle_serious_injury_count': 0,
    'other_death_count': 0,
    'other_serious_injury_count': 0,
    'micromobility_serious_injury_count': 0,
    'micromobility_death_count': 0,
    'severity_killed': False,
    'severity_not_injured': False,
    'severity_unknown': False,
    # 'unit_involved_motor_vehicle_other': 0,
    # 'unit_involved_motorcycle': 0,
    'hour': 13,
    'weekend': 0,
    'hour_cos': -0.965926
}

new_prediction = np.array([vars.values()])

In [6]:
print(new_prediction)

[dict_values([False, 35.0, 0, 0, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, False, False, False, 13, 0, -0.965926])]


# Scale

In [7]:
new_prediction = np.array([list(vars.values())], dtype=float)
new_prediction = scaler.transform(new_prediction)
print(new_prediction)

[[-7.46081871e-02 -8.61017173e-01 -1.62935804e-01 -4.42893547e-01
   4.81622113e+00 -5.11110094e-01 -2.84014209e-01 -4.88638347e-02
  -1.32135153e-01 -1.12080440e-02 -3.12444015e-02 -4.16565417e-02
  -5.94961069e-02 -2.99720892e-02 -7.07378714e-02  0.00000000e+00
  -4.23601439e-03  0.00000000e+00  0.00000000e+00 -7.46081871e-02
  -9.44844109e-01 -2.72299260e-01  2.10513298e-02 -5.94533824e-01
  -3.29044719e+00]]


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


# Results

In [8]:
# unit_involved_motor_vehicle_other = np.array([0])
# new_prediction = np.insert(new_prediction, -3, unit_involved_motor_vehicle_other, axis=1)

# unit_involved_motorcycle = np.array([0])
# new_prediction = np.insert(new_prediction, -3, unit_involved_motorcycle, axis=1)

In [9]:
results = {}
for model in models:
    predictions = models[model].predict(new_prediction)
    result = predictions[0]
    print(f"The {model} model predicted: {result}")

NotFittedError: Pipeline is not fitted yet.

In [ ]:
print(results)

# Visualization